## Librerias

## Modelo time series test 1

In [11]:
import opendatasets as od

import os, glob
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)

import sweetviz as sv

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressor

In [12]:
df_train = pd.read_csv('train.csv', sep=',')
df_test = pd.read_csv('test.csv', sep=',')
df_test_sample = pd.read_csv('test_sample.csv', sep=',')

In [13]:
import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

## 1. Melt the data, stack the series

In [177]:
data = pd.read_csv('train.csv')
#data = data.filter(regex=r'Product|W')
data.head()

,Z_MARCA,Z_GAMA,Z_MODELO,Z_DEPARTAMENTO,Z_PUNTO_VENTA,SEMANA_01,SEMANA_02,SEMANA_03,SEMANA_04,SEMANA_05,SEMANA_06,SEMANA_07,SEMANA_08,SEMANA_09,SEMANA_10,SEMANA_11,SEMANA_12,SEMANA_13,SEMANA_14,SEMANA_15,SEMANA_16,SEMANA_17,SEMANA_18,SEMANA_19,SEMANA_20,SEMANA_21,SEMANA_22,SEMANA_23,SEMANA_24,SEMANA_25,SEMANA_26,SEMANA_27,SEMANA_28,SEMANA_29,SEMANA_30,SEMANA_31,SEMANA_32,SEMANA_33,SEMANA_34,SEMANA_35,SEMANA_36,SEMANA_37,SEMANA_38,SEMANA_39,SEMANA_40,SEMANA_41,SEMANA_42,SEMANA_43,SEMANA_44,SEMANA_45,SEMANA_46,SEMANA_47,SEMANA_48,SEMANA_49,SEMANA_50
0,f223faa96f22916294922b171a2696d868fd1f9129302e...,de88c121a82a06352bf1aaceba20578356408a334ba046...,f0465138ce3c092d78c1e33657fe604564d40cdc8cb196...,591c0a0133cb5fcd00af7bbf046f094256901239749fb7...,d2c888e1a77f2eb0732555cf018c3ca71bbcb32c73778c...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2c...,76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435...,85ac1d5351fa6c551dcaf4e9440939949f59fd6986c5ee...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,99af5fbe4f1ce1a2d5c05d8d50543bb993dd621f259c8e...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,f223faa96f22916294922b171a2696d868fd1f9129302e...,de88c121a82a06352bf1aaceba20578356408a334ba046...,ab82a1d82b6fbd27ffe90900dffa4e8018745ef082fdf2...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,4d9927f9d9ebe9b3742b20cf87b37b72c526f39d160289...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,285075a02b2679248a6b4636c3328bd3097626607c3e43...,8563abec343968034b0624650aed7254081e9e39e6b32e...,3f7d19feb71e55fc12b796a4cda0fbcec00511a039e758...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,2fc9e521d966b9a311c1d3fc70abafa98dd0f37a51c71a...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17...,8563abec343968034b0624650aed7254081e9e39e6b32e...,dbd49c8cda7f4bbbfc2a8b337b5aa79dc8067b46ca1bf5...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,729be1d813198ffecf16f8c581e474cd58ee5aa7d11cdf...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [178]:
data['Product_Code'] = data['Z_MODELO']+data['Z_PUNTO_VENTA']+data['Z_GAMA']
data_base = data.copy()
data_base 

,Z_MARCA,Z_GAMA,Z_MODELO,Z_DEPARTAMENTO,Z_PUNTO_VENTA,SEMANA_01,SEMANA_02,SEMANA_03,SEMANA_04,SEMANA_05,SEMANA_06,SEMANA_07,SEMANA_08,SEMANA_09,SEMANA_10,SEMANA_11,SEMANA_12,SEMANA_13,SEMANA_14,SEMANA_15,SEMANA_16,SEMANA_17,SEMANA_18,SEMANA_19,SEMANA_20,SEMANA_21,SEMANA_22,SEMANA_23,SEMANA_24,SEMANA_25,SEMANA_26,SEMANA_27,SEMANA_28,SEMANA_29,SEMANA_30,SEMANA_31,SEMANA_32,SEMANA_33,SEMANA_34,SEMANA_35,SEMANA_36,SEMANA_37,SEMANA_38,SEMANA_39,SEMANA_40,SEMANA_41,SEMANA_42,SEMANA_43,SEMANA_44,SEMANA_45,SEMANA_46,SEMANA_47,SEMANA_48,SEMANA_49,SEMANA_50,Product_Code
0,f223faa96f22916294922b171a2696d868fd1f9129302e...,de88c121a82a06352bf1aaceba20578356408a334ba046...,f0465138ce3c092d78c1e33657fe604564d40cdc8cb196...,591c0a0133cb5fcd00af7bbf046f094256901239749fb7...,d2c888e1a77f2eb0732555cf018c3ca71bbcb32c73778c...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f0465138ce3c092d78c1e33657fe604564d40cdc8cb196...
1,5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2c...,76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435...,85ac1d5351fa6c551dcaf4e9440939949f59fd6986c5ee...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,99af5fbe4f1ce1a2d5c05d8d50543bb993dd621f259c8e...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,85ac1d5351fa6c551dcaf4e9440939949f59fd6986c5ee...
2,f223faa96f22916294922b171a2696d868fd1f9129302e...,de88c121a82a06352bf1aaceba20578356408a334ba046...,ab82a1d82b6fbd27ffe90900dffa4e8018745ef082fdf2...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,4d9927f9d9ebe9b3742b20cf87b37b72c526f39d160289...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ab82a1d82b6fbd27ffe90900dffa4e8018745ef082fdf2...
3,285075a02b2679248a6b4636c3328bd3097626607c3e43...,8563abec343968034b0624650aed7254081e9e39e6b32e...,3f7d19feb71e55fc12b796a4cda0fbcec00511a039e758...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,2fc9e521d966b9a311c1d3fc70abafa98dd0f37a51c71a...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3f7d19feb71e55fc12b796a4cda0fbcec00511a039e758...
4,df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17...,8563abec343968034b0624650aed7254081e9e39e6b32e...,dbd49c8cda7f4bbbfc2a8b337b5aa79dc8067b46ca1bf5...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,729be1d813198ffecf16f8c581e474cd58ee5aa7d11cdf...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,dbd49c8cda7f4bbbfc2a8b337b5aa79dc8067b46ca1bf5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47168,9fc7c2306f5afb4bfe47feefbd193f245633febbb18171...,76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435...,c2890f116e4a0c94ec2c117957dca34875d5118a80aefa...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,4eec71b0e096a148195feadd5e6e316dff031709400305...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,c2890f116e4a0c94ec2c117957dca34875d5118a80aefa...
47169,9fc7c2306f5afb4bfe47feefbd193f245633febbb18171...,76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435...,c2890f116e4a0c94ec2c117957dca34875d5118a80aefa...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,f100265b56fc39cfd0317e03d0040f9d31514590d8f345...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,c2890f116e4a0c94ec2c117957dca34875d5118a80aefa...
47170,9fc7c2306f5afb4bfe47feefbd193f245633febbb18171...,76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435...,c2890f116e4a0c94ec2c117957dca34875d5118a80aefa...,d6c21b948958417ca98b682a573eb8aa1084b292d32f76...,fb0c21afb8e43d619ff3617e5bd3822e8d7ed4e0a8ecef...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,c2890f116e4a0c94ec2c117957d

In [179]:
##Product code debe ser un ID unico
data['Product_Code'].value_counts().max()

1

In [180]:
data=data.drop(columns=['Z_MARCA','Z_GAMA','Z_MODELO','Z_DEPARTAMENTO','Z_PUNTO_VENTA'])

In [181]:
data_dummy=data_base[['Product_Code','Z_MARCA','Z_GAMA','Z_MODELO','Z_DEPARTAMENTO','Z_PUNTO_VENTA']]

In [182]:
data_dummy2=pd.get_dummies(data=data_dummy,columns=['Z_MARCA','Z_GAMA','Z_DEPARTAMENTO'])

In [185]:
data_dummy3=data_dummy2.drop(columns=['Z_MODELO','Z_PUNTO_VENTA'])

In [191]:
features_2=data_dummy3.columns[1:]

In [183]:
melt = data.melt(id_vars='Product_Code', var_name='Week', value_name='Sales')
melt.head()

,Product_Code,Week,Sales
0,f0465138ce3c092d78c1e33657fe604564d40cdc8cb196...,SEMANA_01,0
1,85ac1d5351fa6c551dcaf4e9440939949f59fd6986c5ee...,SEMANA_01,0
2,ab82a1d82b6fbd27ffe90900dffa4e8018745ef082fdf2...,SEMANA_01,1
3,3f7d19feb71e55fc12b796a4cda0fbcec00511a039e758...,SEMANA_01,0
4,dbd49c8cda7f4bbbfc2a8b337b5aa79dc8067b46ca1bf5...,SEMANA_01,0


In [186]:
melt=pd.merge(melt,data_dummy3,on='Product_Code',how='left')

In [192]:
#melt['Product_Code'] = melt['Product_Code'].str.extract('(\d+)', expand=False).astype(int)
melt['Week'] = melt['Week'].str.extract('(\d+)', expand=False).astype(int)

melt = melt.sort_values(['Week', 'Product_Code'])
melt.head()

,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53
12825,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12818,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,1,0,0,0,0,0,0,0,

In [193]:
melt['Week'].max()

50

### SPLIT DATA

In [194]:
split_point = 40
melt_train = melt[melt['Week'] < split_point].copy()
melt_valid = melt[melt['Week'] >= split_point].copy()

In [195]:
melt_train.head()

,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53
12825,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12818,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,1,0,0,0,0,0,0,0,

### Target

In [196]:
melt_train['sales_next_week'] = melt_train.groupby("Product_Code")['Sales'].shift(-1)


In [197]:
melt_train.head()

,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53,sales_next_week
12825,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0
12818,009e0874cc07b2180b2b2b1039321041a1f42e66693c49

In [198]:
melt_valid['sales_next_week'] = melt_valid.groupby("Product_Code")['Sales'].shift(-1)

In [199]:
melt_train.tail()

,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53,sales_next_week
1837743,ffa11a39a8188306f7952912725f97436e229672141654...,39,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN
1837806,ffa11a39a8188306f7952912725f97436e2296721

In [200]:
melt_train = melt_train.dropna()

In [201]:
melt_train.tail()

,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53,sales_next_week
1790570,ffa11a39a8188306f7952912725f97436e229672141654...,38,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0
1790633,ffa11a39a8188306f7952912725f97436e2296721

### Fundamental Features

#### Lag

In [202]:
melt_train["lag_sales_1"] = melt_train.groupby("Product_Code")['Sales'].shift(1)

In [203]:
melt_train["lag_sales_2"] = melt_train.groupby("Product_Code")['Sales'].shift(2)

In [204]:
melt_train.iloc[12825,0]

'40a38e760557372a93559d29bb37a6d3dc731f6dff4bc9568da22baf9c6dcc7a1970f43775f2258ac2c512384323166b687aa14152129a8e2905f0b342351a5ede88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e'

In [205]:
melt_train[melt_train['Product_Code'] == "40a38e760557372a93559d29bb37a6d3dc731f6dff4bc9568da22baf9c6dcc7a1970f43775f2258ac2c512384323166b687aa14152129a8e2905f0b342351a5ede88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e"].head()

,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53,sales_next_week,lag_sales_1,lag_sales_2
35640,40a38e760557372a93559d29bb37a6d3dc731f6dff4bc9...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,NaN,NaN
82813,40a38e76055737

In [206]:
melt_valid["lag_sales_1"] = melt_valid.groupby("Product_Code")['Sales'].shift(1)

In [207]:
melt_valid["lag_sales_2"] = melt_valid.groupby("Product_Code")['Sales'].shift(2)

#### Diference

In [208]:
melt_train["diff_sales_1"] = melt_train.groupby("Product_Code")['Sales'].diff(1)

In [209]:
melt_train["diff_sales_2"] = melt_train.groupby("Product_Code")['Sales'].diff(2)

In [210]:
melt_train[melt_train['Product_Code'] == "40a38e760557372a93559d29bb37a6d3dc731f6dff4bc9568da22baf9c6dcc7a1970f43775f2258ac2c512384323166b687aa14152129a8e2905f0b342351a5ede88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e"].head()

,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53,sales_next_week,lag_sales_1,lag_sales_2,diff_sales_1,diff_sales_2
35640,40a38e760557372a93559d29bb37a6d3dc731f6dff4bc9...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,Na

In [211]:
melt_valid["diff_sales_1"] = melt_valid.groupby("Product_Code")['Sales'].diff(1)

In [212]:
melt_valid["diff_sales_2"] = melt_valid.groupby("Product_Code")['Sales'].diff(2)

#### Estadisticas descriptiva

In [213]:
melt_train.groupby("Product_Code")['Sales'].rolling(4).mean()

Product_Code                                                                                                                                                                                             
009e0874cc07b2180b2b2b1039321041a1f42e66693c499caf59fc74c0c45801043d83fbb5c4d83c2fb27d8e7db6ece7227740bf416e91b944092eeb8eee5932c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a  12825       NaN
                                                                                                                                                                                                  59998       NaN
                                                                                                                                                                                                  107171      NaN
                                                                                                                                                                        

In [214]:
melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

12825       NaN
59998       NaN
107171      NaN
154344     0.00
201517     0.25
           ... 
1557743    0.00
1604916    0.00
1652089    0.00
1699262    0.00
1746435    0.00
Name: Sales, Length: 1792574, dtype: float64

In [215]:
melt_train["mean_sales_4"] = melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

In [216]:
melt_train["mean_sales_2"] = melt_train.groupby("Product_Code")['Sales'].rolling(2).mean().reset_index(level=0, drop=True)

In [217]:
melt_train[melt_train['Product_Code'] == "40a38e760557372a93559d29bb37a6d3dc731f6dff4bc9568da22baf9c6dcc7a1970f43775f2258ac2c512384323166b687aa14152129a8e2905f0b342351a5ede88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e"].head()


,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53,sales_next_week,lag_sales_1,lag_sales_2,diff_sales_1,diff_sales_2,mean_sales_4,mean_sales_2
35640,40a38e760557372a93559d29bb37a6d3dc731f6dff4bc9...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,

In [218]:
melt_valid["mean_sales_4"] = melt_valid.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

In [219]:
melt_valid["mean_sales_2"] = melt_valid.groupby("Product_Code")['Sales'].rolling(2).mean().reset_index(level=0, drop=True)

In [220]:
melt_valid=melt_valid[~melt_valid['sales_next_week'].isnull()]

#### Metricas

In [221]:
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    #ape[~np.isfinite(ape)] = 0. # VERY questionable
    ape[~np.isfinite(ape)] = 1. # pessimist estimate
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

#### Linea Base

In [222]:
y_pred = melt_train['Sales']
y_true = melt_train['sales_next_week']

In [223]:
mape(y_true, y_pred)

0.9693358823679581

In [224]:
wmape(y_true, y_pred)

0.65553072337782

#### Entrenando el modelo

In [225]:
melt_train.head(5)

,Product_Code,Week,Sales,Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521,Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b,Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5,Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2,Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29,Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa,Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d,Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5,Z_MARCA_53c759c773f2b832ddcc61534bdb919d44e03be212241e7dad49b286afa95340,Z_MARCA_9fc7c2306f5afb4bfe47feefbd193f245633febbb1817113fdf0a768172285b5,Z_MARCA_b870b7809747623def661ded7b0da0e52beddfc50235bbda65f1e8a34350ff7a,Z_MARCA_c0dd56d5b73d72cbd9a3df3f70e0df98ce15417d0f8c3a678280beb763388a4f,Z_MARCA_c25b14ec280222786bd605c2e1d072a98fc18612a2dfe094da1a269e436f4621,Z_MARCA_df853f864c74fa85acd3a25cd0afed68f1f1d0ab249e17cdb25e77569ac595a2,Z_MARCA_f223faa96f22916294922b171a2696d868fd1f9129302eb41a45b2a2ea2ebbfd,Z_MARCA_f41bcffa7463b16f9fea3293a550589718777902b4406c44e3150dad7c47bc3c,Z_MARCA_f4920dac3075feca0bd92a92f6691f50e9f815e5e398f88c188499875b4a5129,Z_GAMA_14a2db592fdae9640e32d665e0b0007aecdf0dcad631fed35a1e60a23bef683a,Z_GAMA_392323a5d3dcd1a8e9076af54e17b493ec11dceab6311d4bbab6044846cbbc3f,Z_GAMA_76df0c6db32d4e04e0ef6a3a6a1e1686677e34308d9435892b1ab5713e57d470,Z_GAMA_8563abec343968034b0624650aed7254081e9e39e6b32ed45171fe31f6b7ee59,Z_GAMA_c4ebc6d4a5832cd9415f906ad03661110c705a72381c8b8b145761d02e2dd23a,Z_GAMA_de88c121a82a06352bf1aaceba20578356408a334ba046a1084d4e50da923c5e,Z_GAMA_f793de205ead5ac302c4a1627829dea41f176b1068b993a32373fc869918374b,Z_DEPARTAMENTO_110820d992b7649baca108cda049eff2fcf6804fd4c989a60e4128406d56b14c,Z_DEPARTAMENTO_1a7f1b5c30851a16a7831c496ed9e3d33e013f3a6e56713a0dcfa57189cee037,Z_DEPARTAMENTO_1e4e6d80f0a8df2a8145eec37f6efe3d0f7909cf05a4da01ac9eb0886dff2122,Z_DEPARTAMENTO_1fdd6772a58aa4eaf8c0055dfc2041085e5fc1e18aae6e9a2ad7db83f2355c5c,Z_DEPARTAMENTO_2170256e37ed495d1b4beceb046f9882b7d3628336fc04b9f6844822bc0f059a,Z_DEPARTAMENTO_28fa9368ededd016d5bd24b5d59fd461840dd1cc4fdaa8e62ed54bbdf51fa42e,Z_DEPARTAMENTO_41e2c31b68bdc286a69459b7899e425343891c59d4ee0cee2d135881a9dc6302,Z_DEPARTAMENTO_588ad85b346966aa3c52a8c80b4da62b8d8e39e6f37f38eb8a3907cba122bbca,Z_DEPARTAMENTO_591c0a0133cb5fcd00af7bbf046f094256901239749fb779b3c5aba2cfa66af7,Z_DEPARTAMENTO_5cd7368176e519fd5ccff4d61db909e869586b350e02f9ca13ff19de8e75c548,Z_DEPARTAMENTO_6375751784407a687081e34abb7c3e44a1d37b2d02ca31b97daf7131f0cc2ce8,Z_DEPARTAMENTO_654cc2e01783fa726c8b5743841202e3f95ae18fb4697f2a56f47cbe3d736434,Z_DEPARTAMENTO_6ff2a97bf9139cd1f16dfd60cb95f37b7a74901738f7e7154d439002ca56bdb1,Z_DEPARTAMENTO_7556b904d2033b7ef193294c2223f4c162c9e6337fc1a37cb5a926c605ab639e,Z_DEPARTAMENTO_89e386538bdee9222240ad020bad45bbdfc465ef98a269fb9a2e43f430737910,Z_DEPARTAMENTO_aa34223f43ece8f311e87687550cc5cd8e90d0e08d8545f3e93da2c4dccbb581,Z_DEPARTAMENTO_b15acf3f3f9ac39f827a8f5e4559b8663be3b48b283dcd373a3dec87a1b3c902,Z_DEPARTAMENTO_b265454a0d9f1a95defdde88e744ba25914f11a13667e63e902cd6dcb197aa34,Z_DEPARTAMENTO_b4881844f3dc91fa8992ca5489c7a0e31f23f1d14744619fd0a71b2d6da79139,Z_DEPARTAMENTO_b4fe46822b8133e110db34fc42f36dc17c825ee69cb7a300fe10119722f801db,Z_DEPARTAMENTO_c128764a760c53a40bf2f3595b8cc89f4f43a8ea977f9d3d872b9f665f0bae2e,Z_DEPARTAMENTO_c6fb05363dbe1210b658f130faf72d57c364e2e98fb5eae38c97f7e0a8482d42,Z_DEPARTAMENTO_d6c21b948958417ca98b682a573eb8aa1084b292d32f760f253ef53da13e5589,Z_DEPARTAMENTO_e664206bf198ba39ebbfebb28817083919e34e7ab81caeeea5d30b980b4f58b6,Z_DEPARTAMENTO_ee5949f8b5ff823b330c243762295ce7ede1a751f35b941253cb473ca63e7d53,sales_next_week,lag_sales_1,lag_sales_2,diff_sales_1,diff_sales_2,mean_sales_4,mean_sales_2
12825,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [164]:
features = ['Sales', 'lag_sales_1', 'diff_sales_1', 'mean_sales_4', 'lag_sales_2', "mean_sales_2"] #'diff_sales_2',

In [226]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(melt_train[features])
ytr = melt_train['sales_next_week']

Xtr
#mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
#mdl.fit(Xtr, ytr)

array([[ 0.        ,  0.80438822, -0.00186948,  0.80942806,  0.80993895,
         0.80345348],
       [ 0.        ,  0.80438822, -0.00186948,  0.80942806,  0.80993895,
         0.80345348],
       [ 0.        ,  0.80438822, -0.00186948,  0.80942806,  0.80993895,
         0.80345348],
       ...,
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [232]:
Xtr_1 = pd.concat([melt_train[features_2] ,pd.DataFrame(Xtr, columns=features)], axis=1).values

mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
mdl.fit(Xtr_1, ytr)

RandomForestRegressor(n_jobs=6, random_state=0)

In [233]:
true_y_train=ytr.values

In [235]:

p_train = mdl.predict(Xtr_1)

In [236]:
p_train

array([0.07622442, 0.10610385, 0.14168582, ..., 0.06123724, 0.05172793,
       0.05733754])

In [237]:
rmse_train = metrics.mean_squared_error(true_y_train,p_train , squared=False)
rmse_train

1.3047683967650887

#### Evaluacion del modelo

In [244]:
melt_valid.isna().sum()

Product_Code                                                                            0
Week                                                                                    0
Sales                                                                                   0
Z_MARCA_0855cd73c3a9dc2c31a7c4dcaecfd53282238c6a457a5c771cbee045c0d9a521                0
Z_MARCA_09c32757a9ca408c829114503523a5e6762dd1ba2b81067ad113837c1b30188b                0
Z_MARCA_1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5                0
Z_MARCA_22ea092e7643557ea91b74dea27589acac28a04fce976c577c422cd3ee1c9dc2                0
Z_MARCA_285075a02b2679248a6b4636c3328bd3097626607c3e43c0c5498258fbfa9f29                0
Z_MARCA_2aca20d3a48b13f1d2ee88e9746a4fb6d2846684084819272d6e5ff8809d62aa                0
Z_MARCA_4b1acc0002585cda1245f0e07c6f58ff0e3b5a0e8d3187fa55d6e0ed43836c1d                0
Z_MARCA_5132f94c2aebce767bd61d9e8f0d4f681d0809ca90cd2cd09be494f221367bb5                0
Z_MARCA_53

In [238]:
melt_valid['sales_next_week'].isnull().sum()

0

In [246]:
Xval = imputer.transform(melt_valid[features])
yval = melt_valid['sales_next_week']

#p = mdl.predict(Xval)

In [253]:
melt_valid.shape

(471730, 59)

In [249]:
Xval.shape

(471730, 6)

In [262]:
Xval_1 = pd.concat([melt_valid[features_2].reset_index(drop=True) ,pd.DataFrame(Xval, columns=features).reset_index(drop=True)], axis=1).values

In [263]:
p = mdl.predict(Xval_1)

In [264]:
mape(yval, p)

0.966747242378495

In [265]:
wmape(yval, p)


0.7999614183305656

In [266]:
metrics.mean_squared_error(yval,p , squared=False)

2.2625189887658705

In [267]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [30, 50, 80, 100],
    'max_features': [2, 3, 5],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100]
}
# Create a based model
rf = RandomForestRegressor(random_state=0)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(Xtr_1, ytr)

In [ ]:
best_grid = grid_search.best_estimator_ 

p_best_grid = best_grid.predict(Xval_1)

#### Extencion del modelon prediccion n-rezagos

In [161]:
melt_train['sales_next_next_week'] = melt_train.groupby("Product_Code")['Sales'].shift(-2)
melt_valid['sales_next_next_week'] = melt_valid.groupby("Product_Code")['Sales'].shift(-2)

In [162]:
melt_train = melt_train.dropna(subset=['sales_next_week','sales_next_next_week'])

In [163]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(melt_train[features])
ytr = melt_train[['sales_next_week', 'sales_next_next_week']]

mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
mdl.fit(Xtr, ytr)

KeyboardInterrupt: 

In [ ]:
Xval = imputer.transform(melt_valid[features])
yval = melt_valid[['sales_next_week', 'sales_next_next_week']]

p = mdl.predict(Xval)

In [ ]:
mape(yval, p)

sales_next_week         0.961358
sales_next_next_week    0.969897
dtype: float64

In [ ]:
wmape(yval, p)

sales_next_week         0.782743
sales_next_next_week    0.900575
dtype: float64

In [ ]:
yval["sales_next_week"].values

array([ 0.,  0.,  0., ..., nan, nan, nan])

In [ ]:
p[:,0]

array([0.12182877, 0.12182877, 0.12182877, ..., 0.05014538, 0.05014538,
       0.05014538])

In [ ]:
# %% METRICAS

rmse_t= metrics.mean_squared_error(y_train,pred_train , squared=False)
rmse_val = metrics.mean_squared_error(yval["sales_next_week"],p[:,0], squared=False)

KeyError: 0